# Assignment 13: Dash App
Dash App\
Ismail Abdo Elmaliki\
CS 502 - Predictive Analytics\
Capitol Technology University\
Professor Frank Neugebauer\
March 30, 2022

## Table of Contents
*Import Dependencies & Setup Model*

*Setup App*

*Setup Numerical Features*

*Setup Categorical Features*

*Group Categorical & Numerical Features*

*Setup Prediction Function*

*Setup App Layout*

*Setup Callback & Corresponding Function*

*Conclusion*

## Import Dependencies & Setup Model
Keep in mind these files were generated by running the `diamonds-model-training.py` file first, which generates the pca, diamond-prices-model and scaler.

In [90]:
import os
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from keras.models import load_model
import joblib
import numpy as np
import plotly.graph_objs as go

model = load_model('diamond-prices-model.h5')
pca = joblib.load('pca.joblib')
scaler = joblib.load('scaler.joblib')


## Setup App
Include a theme to make our app look that much better.

In [92]:
app = dash.Dash(__name__)
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

## Setup Numerical Features
Here we're setting up the heading and input of all numerical features for our diamond-prices model. Specifically the following features:
- carat
- depth
- table
- x
- y
- z

In [93]:
input_carat = dcc.Input(id='carat', type='numeric', value=0.7)
div_carat = html.Div(
    children=[html.H3('Carat:'), input_carat],
    className='four columns'
)

input_depth = dcc.Input(id='depth', placeholder='', type='numeric', value=60)
div_depth = html.Div(
    children=[html.H3('Depth:'), input_depth],
    className='four columns'
)

input_table = dcc.Input(
    id='table',
    placeholder='',
    type='numeric',
    value=60
)
div_table = html.Div(
    children=[html.H3('Table:'), input_table],
    className='four columns'
)

input_x = dcc.Input(
    id='x',
    placeholder='',
    type='numeric',
    value=5
)
div_x = html.Div(
    children=[html.H3('X:'), input_x],
    className='four columns'
)

input_y = dcc.Input(
    id='y',
    placeholder='',
    type='numeric',
    value=5
)
div_y = html.Div(
    children=[html.H3('Y:'), input_y],
    className='four columns'
)

input_z = dcc.Input(
    id='z',
    placeholder='',
    type='numeric',
    value=5
)
div_z = html.Div(
    children=[html.H3('Z:'), input_z],
    className='four columns'
)

## Setup Categorical Features
Setting up titles and dropdowns for the following categorical features:
- cut
- color
- clarity

In [94]:
cut_values = ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']
cut_options = [ {'label': x, 'value': x} for x in cut_values ]
input_cut = dcc.Dropdown(
    id='cut',
    options = cut_options,
    value = 'Ideal'
)
div_cut = html.Div(
    children=[html.H3('Cut:'), input_cut],
    className='four columns'
)

color_values = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
color_options = [ {'label': x, 'value': x} for x in color_values ]
input_color = dcc.Dropdown(
    id='color',
    options = color_options,
    value = 'G'
)
div_color = html.Div(
    children=[html.H3('Color:'), input_color],
    className='four columns'
)

clarity_values = ['I1', 'IF', 'SI1', 'VS1', 'VS2', 'VVS1', 'VVS2']
clarity_options = [ {'label': x, 'value': x} for x in clarity_values ]
input_clarity = dcc.Dropdown(
    id='clarity',
    options = clarity_options,
    value = 'SI1'
)
div_clarity = html.Div(
    children=[html.H3('Clarity:'), input_clarity],
    className='four columns'
)

## Group Categorical & Numerical Features
Now that we've setup all titles and inputs for numerical and categorical features, it's best to group them together. That way they can be added to the app layout easily.

In [100]:
div_numerical = html.Div(
    children=[div_carat, div_depth, div_table],
    className='row'
)

div_dimensions = html.Div(
    children=[div_x, div_y, div_z],
    className='row'
)

div_categorical = html.Div(
    children=[div_cut, div_color, div_clarity],
    className='row'
)

## Setup Prediction Function
Included in the function's parameters are the input features, which will be used to return predicted diamond price that will display to the user. For now though this is only the logic for returning a predicted price. Eventually we'll get to how the price is displayed via our Dash app.

In [96]:
def get_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    cols = [
        'carat', 'depth', 'table', 
        'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 
        'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 
        'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 
        'dim_index'
    ]

    cut_dict = { x: 'cut_' + x for x in cut_values[1:] }
    color_dict = { x: 'color_' + x for x in color_values[1:] }
    clarity_dict = { x: 'clarity_' + x for x in clarity_values[1:] }

    df = pd.DataFrame(data= np.zeros( (1, len(cols)) ), columns=cols)
    df.loc[0, 'carat'] = carat
    df.loc[0, 'depth'] = depth
    df.loc[0, 'table'] = table

    dims_df = pd.DataFrame(data=[[x, y, z]], columns=['x', 'y', 'z'])
    df.loc[0, 'dim_index'] = pca.transform(dims_df).flatten()[0]

    if cut != 'Fair':
        df.loc[0, cut_dict[cut]] = 1
    
    if color != 'D':
        df.loc[0, color_dict[color]] = 1

    if clarity != 'I1':
        df.loc[0, clarity_dict[clarity]] = 1
    
    numerical_features = ['carat', 'depth', 'table', 'dim_index']
    df.loc[:, numerical_features] = scaler.transform(df.loc[:, numerical_features])

    prediction = model.predict(df.values).flatten()

    prediction = np.exp(prediction)

    return int(prediction)


## Setup App Layout
Now it's time to setup our Dash app's layout. Included are the following:
- H1 header
- H2 header
- numerical, dimensions and categorical groups (which contain all feature inputs)
- setup how output (predicted price) will be displayed

In [97]:
app.layout = html.Div([
    html.H1('IDR Predict diamond prices'),
    html.H2('Enter the diamond characteristics to get the predicted price'),
    html.Div(
        children=[
            div_numerical,
            div_dimensions,
            div_categorical
        ]
    ),
    html.H1(
        id='output',
        style = {'margin-top': '50px', 'text-align': 'center'}
    )
])

## Setup Callback & Corresponding Function
As the user enters input, we'll need to have a way to update the output everytime. Hence the importance of callbacks to ensure continual updates.

Here the predictors are specified (the input features). From there, we include a corresponding function that will call the `get_prediction` function then return the predicted price which is then displayed to the user.

Last but not least, we'll need to run our app!

In [98]:
predictors = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color', 'clarity']
@app.callback(
    Output('output', 'children'),
    [Input(x, 'value') for x in predictors]
)
def show_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    pred = get_prediction(carat, depth, table, x, y, z, cut, color, clarity)
    print('Actual prediction:', pred)
    return str('Predicted Price: {:,}'.format(pred))

if __name__ == '__main__':
    app.run_server(debug=False)

## Conclusion
Our dash app is successfully setup! Within this app we've been able to do the following:
- Import and setup our model
- Setup layout dependencies for all input features
- Setup a function that returns the predicted diamond price
- Setup our app layout
- Setup callbacks that will continually update the output based on feature input changes